## Instruccions

Introduïu xlsx generat per l'app de comprovació de dominis (whois) o genereu xlsx amb el llsitat de dominis .eu que es volen processar (amb capçalera).


In [1]:
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import time

# Carregar l'arxiu Excel
print("Carrega l'arxiu Excel:")
arxiu = files.upload()

Carrega l'arxiu Excel:


Saving dades - 2024-03-21T140802.597.xlsx to dades - 2024-03-21T140802.597.xlsx


In [2]:
nom_arxiu = list(arxiu.keys())[0]  # Obtenir el nom de l'arxiu
full_data = pd.read_excel(nom_arxiu)

full_data = pd.DataFrame(full_data[full_data.iloc[:, 0].str.endswith('.eu')])

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_data(domini):
    try:
        response = requests.get('https://whois.eurid.eu/en/search/?domain='+domini)
        response.raise_for_status()
        html = response.text
        return extract_data(html, domini)
    except Exception as e:
        print('Error al obtenir el codi HTML:', e)

def extract_data(html, domini):
    soup = BeautifulSoup(html, 'html.parser')
    data = {}

    # Extraure secció "General"
    general_section = soup.find('section', id='section-general')
    if general_section:
        general_data = {}
        general_table = general_section.find('dl', class_='row')
        rows = general_table.find_all(['dt', 'dd'])
        for i in range(0, len(rows), 2):
            key = "General. " + rows[i].get_text().strip()
            value = rows[i+1].get_text().strip()
            if value:
              general_data[key] = value
        data.update(general_data)

    creacioGPG = {}
    key = "Data creació GPG"
    value = full_data[full_data.iloc[:, 0] == domini]["DataCreacioGPG"].iloc[0]
    creacioGPG[key] = value
    data.update(creacioGPG)

    vencimentGPG = {}
    key = "Data venciment GPG"
    value = full_data[full_data.iloc[:, 0] == domini]["DataVencimentGPG"].iloc[0]
    vencimentGPG[key] = value
    data.update(vencimentGPG)

    titularGPG = {}
    key = "Titular GPG"
    value = full_data[full_data.iloc[:, 0] == domini]["Titular"].iloc[0]
    titularGPG[key] = value
    data.update(titularGPG)

    # Extraure secció "Registrant"
    registrant_section = soup.find('dl', {'class': 'row registrant-contact', 'id': 'registrant'})
    if registrant_section:
        registrant_data = {}
        registrant_table = registrant_section.find_all(['dt', 'dd'])
        for i in range(0, len(registrant_table), 2):
            key = "Titular. " + registrant_table[i].get_text().strip()
            value = registrant_table[i+1].get_text().strip()
            if value:
              registrant_data[key] = value
        data.update(registrant_data)

    # Extraure secció "Technical contact"
    technical_contact_section = soup.find('div', {'class': 'tab-pane fade show active d-print-block opacity-print-100', 'id': 'tech-tab-pane-1'})
    if technical_contact_section:
        technical_contact_data = {}
        technical_contact_table = technical_contact_section.find('dl', class_='row')
        if technical_contact_table:
            rows = technical_contact_table.find_all(['dt', 'dd'])
            for i in range(0, len(rows), 2):
                key = "Contacte tècnic. " + rows[i].get_text().strip()
                value = rows[i+1].get_text().strip()
                if value:
                  technical_contact_data[key] = value
        data.update(technical_contact_data)

    # Extraure la taula dels servidors de noms
    nameservers_section = soup.find('dl', id='nameservers')
    if nameservers_section:
        nameservers_data = {}
        nameservers_table = nameservers_section.find_all(['dt', 'dd'])
        if nameservers_table:
            for i in range(0, len(nameservers_table), 2):
                key = "Servidors. " + nameservers_table[i].get_text().strip()
                value = nameservers_table[i+1].get_text().strip()
                if value:
                  nameservers_data[key] = value
        data.update(nameservers_data)

    # Eliminar els salts de línia de totes les dades
    for key, value in data.items():
        data[key] = ' '.join(value.replace('\n', '').split())

    return data


In [6]:
print("Total dominis:", len(full_data))

dades = []  # Llista per emmagatzemar les dades

for index, codi in enumerate(full_data.iloc[:, 0], 1):
    print(codi)
    # Crida a la funció fetch_data()
    dades.append(fetch_data(codi))
    time.sleep(30)

    # Comprovar si l'índex és múltiple de 5
    if index % 5 == 0:
        print("Esperant 120 segons...")
        time.sleep(120)  # Espera de 120 segons

# Crear DataFrame de Pandas
df = pd.DataFrame(dades)  # Assignem una única fila com a índex
df


Total dominis: 3
heptromic.eu
ict-arcfire.eu
transcoloncan.eu


,General. Domain name,General. Status,General. Registered,General. Registrar,Data creació GPG,Data venciment GPG,Titular GPG,Titular. Organisation,Titular. Language,Servidors. Name server #1,Servidors. Name server #2,Servidors. Name server #3,Contacte tècnic. Organisation,Contacte tècnic. Language,Servidors. Name server #4
0,heptromic.eu,Registered,08 January 2020,Key-Systems GmbH,23-11-2010,22-11-2023,Consorci Institut d'Investigacions Biomèdiques...,Plazachemical,English,ns3.rrpproxy.net,ns2.rrpproxy.net,ns1.rrpproxy.net,NaN,NaN,NaN
1,ict-arcfire.eu,Registered,20 January 2024,IONOS SE,10-12-2015,08-12-2023,"Fundació Privada I2CAT, Internet i Innovació D...",tramao GmbH,German,ns1034.ui-dns.biz,ns1052.ui-dns.org,ns1062.ui-dns.de,1&1 Internet SE,English,ns1098.ui-dns.com
2,transcoloncan.eu,Registered,25 November 2023,Porkbun LLC,30-11-2018,30-11-2023,Consorci Institut d'Investigacions Biomèdiques...,NaN,English,ns2.nicsell.com,ns1.nicsell.com,NaN,Porkbun LLC,English,NaN


In [7]:
from datetime import date
avui = date.today().strftime("%Y%m%d")

In [8]:
# Guardar el DataFrame en un fitxer Excel
df.to_excel(avui+"_comprovacio_eurid.xlsx", index=False)

# Descarregar l'arxiu Excel
files.download(avui+"_comprovacio_eurid.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>